In [62]:
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import json

In [63]:
data_cords = pd.read_csv('data_cords.csv')
data_cords.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [64]:
data_cords.drop(columns=['Unnamed: 0'], inplace=True)

In [65]:
data_cords['Neighbourhood'].value_counts()

The Beaches                                                                1
The Junction North ,Runnymede                                              1
Woburn                                                                     1
Studio District                                                            1
Maryvale ,Wexford                                                          1
                                                                          ..
Kingsview Village ,Martin Grove Gardens ,Richview Gardens ,St. Phillips    1
Brockton ,Exhibition Place ,Parkdale Village                               1
Westmount                                                                  1
Woodbine Heights                                                           1
East Toronto                                                               1
Name: Neighbourhood, Length: 103, dtype: int64

In [66]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data_cords['Borough'].unique()),
        data_cords.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [67]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [68]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(data_cords['Latitude'], data_cords['Longitude'], data_cords['Borough'], data_cords['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


In [69]:
LIMIT = 100

CLIENT_ID = 'T4Z505EWK323YATEMXLUKVBTOLJ2G2GC5L3M3H0AOX3PKOBB' 
CLIENT_SECRET = '5TVDN1050RW43HMIS2ZZ3NJXSDDHH1TP0DQJNQBMAI2DBGTP' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: T4Z505EWK323YATEMXLUKVBTOLJ2G2GC5L3M3H0AOX3PKOBB
CLIENT_SECRET:5TVDN1050RW43HMIS2ZZ3NJXSDDHH1TP0DQJNQBMAI2DBGTP


In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names=data_cords['Neighbourhood'],
                                   latitudes=data_cords['Latitude'],
                                   longitudes=data_cords['Longitude'],
                                  )

Rouge ,Malvern
Highland Creek ,Rouge Hill ,Port Union
Guildwood ,Morningside ,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park ,Ionview ,Kennedy Park
Clairlea ,Golden Mile ,Oakridge
Cliffcrest ,Cliffside ,Scarborough Village West
Birch Cliff ,Cliffside West
Dorset Park ,Scarborough Town Centre ,Wexford Heights
Maryvale ,Wexford
Agincourt
Clarks Corners ,Sullivan ,Tam O'Shanter
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview ,Henry Farm ,Oriole
Bayview Village
Silver Hills ,York Mills
Newtonbrook ,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park ,Don Mills South
Bathurst Manor ,Downsview North ,Wilson Heights
Northwood Park ,York University
CFB Toronto ,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens ,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West ,

In [72]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge ,Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [73]:
toronto_venues.shape

(2147, 7)

In [74]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide ,King ,Richmond",95,95,95,95,95,95
Agincourt,4,4,4,4,4,4
"Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East",3,3,3,3,3,3
"Albion Gardens ,Beaumond Heights ,Humbergate ,Jamestown ,Mount Olive ,Silverstone ,South Steeles ,Thistletown",11,11,11,11,11,11
"Alderwood ,Long Branch",10,10,10,10,10,10
...,...,...,...,...,...,...
Willowdale West,7,7,7,7,7,7
Woburn,4,4,4,4,4,4
"Woodbine Gardens ,Parkview Hill",12,12,12,12,12,12


In [75]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide ,King ,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.031579,...,0.0,0.0,0.010526,0.0,0.0,0.0,0.0,0.0,0.0,0.010526
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Alderwood ,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
97,"Woodbine Gardens ,Parkview Hill",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.0,0.000000


In [77]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,King ,Richmond----
           venue  freq
0    Coffee Shop  0.08
1           Café  0.05
2     Restaurant  0.04
3          Hotel  0.03
4  Deli / Bodega  0.03


----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3         Chinese Restaurant  0.25
4                     Market  0.00


----Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2          Playground  0.33
3      Medical Center  0.00
4  Miscellaneous Shop  0.00


----Albion Gardens ,Beaumond Heights ,Humbergate ,Jamestown ,Mount Olive ,Silverstone ,South Steeles ,Thistletown----
                  venue  freq
0         Grocery Store  0.18
1   Fried Chicken Joint  0.09
2        Discount Store  0.09
3           Coffee Shop  0.09
4  Fast Food Restaurant  0.09


----Alderwood ,Long Branch----
            venue  freq
0     P

                 venue  freq
0          Pizza Place  0.50
1                  Pub  0.25
2  Japanese Restaurant  0.25
3          Yoga Studio  0.00
4       Medical Center  0.00


----Guildwood ,Morningside ,West Hill----
                venue  freq
0      Medical Center  0.14
1        Intersection  0.14
2   Electronics Store  0.14
3  Mexican Restaurant  0.14
4      Breakfast Spot  0.14


----Harbord ,University of Toronto----
                 venue  freq
0                 Café  0.15
1            Bookstore  0.06
2           Restaurant  0.06
3  Japanese Restaurant  0.06
4   Italian Restaurant  0.06


----Harbourfront ,Regent Park----
            venue  freq
0     Coffee Shop  0.15
1            Park  0.06
2          Bakery  0.06
3             Pub  0.06
4  Breakfast Spot  0.04


----Harbourfront East ,Toronto Islands ,Union Station----
                venue  freq
0         Coffee Shop  0.13
1            Aquarium  0.05
2                Café  0.04
3               Hotel  0.04
4  Italian Restaura

               venue  freq
0               Park   0.2
1              Trail   0.2
2                Pub   0.2
3  Health Food Store   0.2
4             Lounge   0.0


----The Beaches West ,India Bazaar----
                  venue  freq
0  Fast Food Restaurant  0.09
1        Sandwich Place  0.09
2           Pizza Place  0.09
3          Liquor Store  0.04
4            Steakhouse  0.04


----The Danforth West ,Riverdale----
                venue  freq
0    Greek Restaurant  0.19
1         Coffee Shop  0.07
2  Italian Restaurant  0.07
3      Ice Cream Shop  0.05
4           Bookstore  0.05


----The Junction North ,Runnymede----
                      venue  freq
0             Grocery Store  0.25
1                  Bus Line  0.25
2               Pizza Place  0.25
3                   Brewery  0.25
4  Mediterranean Restaurant  0.00


----The Kingsway ,Montgomery Road ,Old Mill North----
                venue  freq
0                Park   0.5
1               River   0.5
2  Light Rail Station   0.

In [78]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Restaurant,Gym,Hotel,Clothing Store,Deli / Bodega,American Restaurant,Thai Restaurant,Sushi Restaurant
1,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Chinese Restaurant,Discount Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Coffee Shop,Park,Playground,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
3,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",Grocery Store,Coffee Shop,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Sandwich Place,Pharmacy,Discount Store,Japanese Restaurant
4,"Alderwood ,Long Branch",Pizza Place,Skating Rink,Pharmacy,Sandwich Place,Dance Studio,Pub,Athletics & Sports,Coffee Shop,Gym,Comfort Food Restaurant


In [80]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [81]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data_cords

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353,0.0,Fast Food Restaurant,Women's Store,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,0.0,History Museum,Bar,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Curling Ice
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,0.0,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Bank,Women's Store,Dessert Shop,Deli / Bodega
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Insurance Office,Korean Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dumpling Restaurant,Curling Ice,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Bakery,Bank,Athletics & Sports,Gas Station,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Dessert Shop,Dance Studio


In [82]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [83]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0.0,Fast Food Restaurant,Women's Store,Cupcake Shop,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
1,Scarborough,0.0,History Museum,Bar,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Curling Ice
2,Scarborough,0.0,Electronics Store,Mexican Restaurant,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Bank,Women's Store,Dessert Shop,Deli / Bodega
3,Scarborough,0.0,Coffee Shop,Insurance Office,Korean Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dumpling Restaurant,Curling Ice,Discount Store
4,Scarborough,0.0,Bakery,Bank,Athletics & Sports,Gas Station,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Thai Restaurant,Dessert Shop,Dance Studio
...,...,...,...,...,...,...,...,...,...,...,...,...
97,North York,0.0,Food Service,Baseball Field,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Women's Store,Electronics Store
99,Etobicoke,0.0,Pizza Place,Discount Store,Sandwich Place,Intersection,Chinese Restaurant,Coffee Shop,Distribution Center,Dog Run,Doner Restaurant,Cupcake Shop
100,Etobicoke,0.0,Sandwich Place,Bus Line,Mobile Phone Shop,Pizza Place,Women's Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
101,Etobicoke,0.0,Grocery Store,Coffee Shop,Beer Store,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Sandwich Place,Pharmacy,Discount Store,Japanese Restaurant


In [84]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1.0,Coffee Shop,Park,Playground,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
23,North York,1.0,Park,Bank,Convenience Store,Women's Store,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
25,North York,1.0,Park,Food & Drink Shop,Bus Stop,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store
30,North York,1.0,Business Service,Airport,Park,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant
40,East York,1.0,Coffee Shop,Park,Metro Station,Convenience Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
44,Central Toronto,1.0,Park,Bus Line,Swim School,Women's Store,Dessert Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Diner
50,Downtown Toronto,1.0,Park,Trail,Playground,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store
73,York,1.0,Hockey Arena,Park,Field,Trail,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Donut Shop,Cuban Restaurant
74,York,1.0,Park,Women's Store,Pool,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
79,North York,1.0,Construction & Landscaping,Park,Bakery,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store


In [85]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Playground,Convenience Store,Diner,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store,Cupcake Shop
48,Central Toronto,2.0,Playground,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Women's Store,Cuban Restaurant


In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3.0,Garden,Women's Store,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Cuban Restaurant


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
72,North York,4.0,Pizza Place,Japanese Restaurant,Pub,Women's Store,Dim Sum Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop
96,North York,4.0,Pizza Place,Women's Store,Diner,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store
